In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import bokeh as bk

import panel as pn
import panel.widgets as pnw
pn.extension()



#from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Range1d
#from bokeh.models import TableColumn, DataTable, HoverTool
#from bokeh.models.glyphs import Text, Rect
#from bokeh.layouts import gridplot, column


#from Bio.Seq import Seq
#from Bio.SeqRecord import SeqRecord
#from Bio.Align import MultipleSeqAlignment
#from Bio import AlignIO, SeqIO


#from Bio.Seq import Seq
#from Bio.SeqRecord import SeqRecord
#from Bio.Align import MultipleSeqAlignment
#from Bio import AlignIO, SeqIO

#import os, io, random
#import string

#from IPython.display import HTML
#import panel as pn
#import panel.widgets as pnw
#pn.extension()

#import bokeh
#from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Range1d
#from bokeh.models.glyphs import Text, Rect
#from bokeh.layouts import gridplot, column




In [17]:
### Load the chimeric library
chimeric_library = pd.read_csv("Data/chimeric_lib_representatives_variant_description_top20_msa.csv", header = None)
#chimeric_library = pd.read_csv("Data/chimeric_lib_representatives_variant_description.csv", header = None)

### Create a dictionary with chimeric_name:[serotype ids]
variant_description_dictionary = {}
for chimeric_library_item in chimeric_library[0]:
    
    chimeric_library_item_name = chimeric_library_item.split()[0]
    chimeric_library_item_parents = np.array(chimeric_library_item.split()[1:]).astype("int")
    
    variant_description_dictionary[chimeric_library_item_name] = chimeric_library_item_parents

    

serotype_names = ["No alignment", "AAV1", "AAV2", "AAV3", "AAV4", "AAV5", "AAV6", 
                  "AAV7", "AAV8", "AAV9", "AAV10", "AAV11", "AAV12", "AAV13", 
                  "AAVrh8", "AAVrh10", "AAVrh32", "Multiple alignments", "gap"]

serotype_colors = sns.color_palette("RdBu_r", len(serotype_names)).as_hex()


In [18]:

def view_alignment(serotype_dictionary, include_text = False, plot_width = 800, max_num_x = 100):
    
    ### seqs: Serotype list
    ### keys: Library name
    seqs = np.array(list(variant_description_dictionary.values()))
    keys = np.array(list(variant_description_dictionary.keys()))
    
    N = len(seqs[0])
    S = len(seqs)    
    width = 1.
    
    tmp = [i for s in seqs for i in s]
    colors = [serotype_colors[i] for i in tmp]
    serotypes = [serotype_names[i] for i in tmp]
    
    x = np.arange(1, N + 1)
    y = np.arange(0, S, 1)    
    xx, yy = np.meshgrid(x, y)
    
    gx = xx.flatten()
    gy = yy.flatten()
    recty = gy + .1
    h = 1/S
    
    plot_height = len(seqs)*15 + 50
    x_range = bk.models.Range1d(0, N + 1, bounds = 'auto')
    if N > max_num_x:
        viewlen = max_num_x
    else:
        viewlen = N
    view_range = (0, viewlen) 
    
    
    
    source = bk.models.ColumnDataSource(dict(x=gx, y=gy, recty=recty, colors = colors, serotypes = serotypes))
    
    p = bk.plotting.figure(title = None, 
               plot_width = plot_width, plot_height = plot_height, 
               x_range = view_range, y_range = keys, tools = "xwheel_zoom, xpan, reset", 
               min_border = 0, toolbar_location = 'below')
    
    if include_text == True:
        glyph = bk.models.glyphs.Text(x="x", y="y", text="text", 
                                   text_align='center',
                                   text_color="black", 
                                   text_font="monospace",
                                   text_font_size=fontsize)
    elif include_text == False:
        glyph = bk.models.glyphs.Text(x="x", y="y", text="", 
                                   text_align='center',
                                   text_color="black", 
                                   text_font="monospace")
        
    rects = bk.models.glyphs.Rect(x = "x", y = "recty",  
                 width = 1., height = 1., 
                 fill_color = "colors", 
                 line_color = None, fill_alpha = 1.)
    
    p.add_glyph(source, glyph)
    p.add_glyph(source, rects)
  
    p.grid.visible = False
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.minor_tick_line_width = 0
    p.yaxis.major_tick_line_width = 0
    
    
    tooltips = [("serotype", "@serotypes"),]
    p.add_tools(bk.models.HoverTool(tooltips=tooltips))
    
    return p









In [19]:
p = view_alignment(variant_description_dictionary, include_text = False, plot_width = 800, max_num_x = 2500)
pn.pane.Bokeh(p)

Bokeh(Figure)

In [20]:
color_source = bk.models.ColumnDataSource(dict(name = serotype_names, colors = serotype_colors))


columns = [
    bk.models.TableColumn(field='name', title='serotype_names'),
    bk.models.TableColumn(field='colors', title='colors')
    ]

serotype_table = bk.models.DataTable(source = color_source, columns = columns, fit_columns = False)

pn.pane.Bokeh(serotype_table)


Bokeh(DataTable)